<a href="https://colab.research.google.com/github/AbeerAbuZayed/Machine-Learning-projects/blob/master/text_generation_arabic_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">

  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1ddz4bNhewDHTmG1L3a0_zSEvsZ-qDxaG"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>

</table>

This tutorial demonstrates how to generate text using a character-based RNN.


## Setup

### Import TensorFlow and other libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

### Download the aljazeera.net dataset

Change the following line to run this code on your own data.

In [0]:
filename= 'aljazeera.net_20190419_titles.txt'
URL= "https://github.com/motazsaad/NLP-ICTS6361/raw/master/corpus/aljazeera.net_20190419_titles.txt"

path_to_file = tf.keras.utils.get_file(filename, URL)

### Read the data

First, look in the text:

In [0]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 3355353 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

قطر تشتري حصة بشركة شل
مجلس الأمن يبحث طلب فلسطين
بلاتر: العقوبة ضد السودان عادلة
العراق يلغي صفقة سلاح مع روسيا
جدل بمصر حول قرارات مرسي
غوغل وديل في صفقة بمليار دولار
مصر تهزم زامبيا وتحتفظ بالأمل
4 أغذية عليك تجنبها لترتاح من غازات البطن‬
أميركا و


In [0]:
# The unique characters in the  file
vocab = sorted(set(text)) #28 chars + ?! "" <....
print ('{} unique characters'.format(len(vocab)))

123 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '%' :   5,
  '(' :   6,
  ')' :   7,
  '+' :   8,
  ',' :   9,
  '-' :  10,
  '.' :  11,
  '/' :  12,
  '0' :  13,
  '1' :  14,
  '2' :  15,
  '3' :  16,
  '4' :  17,
  '5' :  18,
  '6' :  19,
  ...
}


In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'قطر تشتري حصة' ---- characters mapped to int ---- > [ 97  91  85   1  78  88  78  85 105   1  81  89  77]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

ق
ط
ر
 
ت


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'قطر تشتري حصة بشركة شل\nمجلس الأمن يبحث طلب فلسطين\nبلاتر: العقوبة ضد السودان عادلة\nالعراق يلغي صفقة سل'
'اح مع روسيا\nجدل بمصر حول قرارات مرسي\nغوغل وديل في صفقة بمليار دولار\nمصر تهزم زامبيا وتحتفظ بالأمل\n4 أ'
'غذية عليك تجنبها لترتاح من غازات البطن\u202c\nأميركا وساعة الحقيقة في مصر\nاستئناف خط كركوك جيهان النفطي\nإخو'
'ان مصر يرفعون عدد مرشحيهم\nثقب الأوزون قد يختفي في عام 2050\nمشهد مهيب في وداع وديع الصافي\nتقدم هام يحل'
' أسرار الميكروبات\nتجميد البويضات بغرض الحمل.. حل مكلف وغير مضمون\nجينوار.. قرية خاصة بالنساء في شمال س'


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'قطر تشتري حصة بشركة شل\nمجلس الأمن يبحث طلب فلسطين\nبلاتر: العقوبة ضد السودان عادلة\nالعراق يلغي صفقة س'
Target data: 'طر تشتري حصة بشركة شل\nمجلس الأمن يبحث طلب فلسطين\nبلاتر: العقوبة ضد السودان عادلة\nالعراق يلغي صفقة سل'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 97 ('ق')
  expected output: 91 ('ط')
Step    1
  input: 91 ('ط')
  expected output: 85 ('ر')
Step    2
  input: 85 ('ر')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 78 ('ت')
Step    4
  input: 78 ('ت')
  expected output: 88 ('ش')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab) #65

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),                         
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'), 
    tf.keras.layers.LSTM(rnn_units, 
                          return_sequences=True,
                          stateful=True, recurrent_initializer='glorot_uniform'),                     
    
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM( batch_input_shape=[batch_size, None])),                                     
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 123) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           31488     
_________________________________________________________________
gru_6 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
gru_7 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense_3 (Dense)              (64, None, 123)           126075    
Total params: 18,786,171
Trainable params: 18,786,171
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [0]:
sampled_indices

array([ 64,  95,  32,  22,  51,  14,  46,  84,  48,  65,  13,  65,  22,
        41, 109, 116,  94, 106, 115,  64,  81,  99,  18,  41,  12,  12,
       121, 109,  35,  11,  12,  56, 107, 108,  33,  97,  85,  14,  85,
        91,  62, 100, 120,  18, 106,  80, 113,  39,  65,  33,   9,  80,
        70,   2,  47,  55,  59,  64,  59,  96, 115,  42,  32,  67, 113,
        44,  47, 108,  39,  50,  56,  71, 109,  93,  88, 113, 109,  33,
         9,  20, 118,  20,  60,   9,  27,   7,  12, 101, 110, 104,  22,
        17,  55, 113,  60,  75,  41,  15,  12,  70])

Decode these to see the text predicted by this untrained model:

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'بعودة وشيكة لصالح لليمن\nإخطار 53 عائلة مقدسية بالهدم\nثوار مصر يدعون لمليونية إنقاذ\nقتلى سوريون بسبب '

Next Char Predictions: 
 'xـH9f1Yذay0y9Sَ٥غً٤xحل5S//\u202bَL./mٌٍIقر1رطuم’5ًجْPyI,جآ!_lrxrف٤TH،ْW_ٍPemأَعشَْI,7\u200e7s,B)/نُى94lْsاS2/آ'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 123)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8120556


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [0]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=20

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
519/519 [==============================] - 265s 510ms/step - loss: 2.1223
Epoch 2/20
519/519 [==============================] - 261s 502ms/step - loss: 1.4988
Epoch 3/20
519/519 [==============================] - 260s 501ms/step - loss: 1.3332
Epoch 4/20
519/519 [==============================] - 260s 501ms/step - loss: 1.1976
Epoch 5/20
519/519 [==============================] - 260s 501ms/step - loss: 1.0755
Epoch 6/20
519/519 [==============================] - 261s 502ms/step - loss: 0.9672
Epoch 7/20
519/519 [==============================] - 261s 502ms/step - loss: 0.8785
Epoch 8/20
519/519 [==============================] - 261s 502ms/step - loss: 0.8123
Epoch 9/20
519/519 [==============================] - 261s 503ms/step - loss: 0.7657
Epoch 10/20
519/519 [==============================] - 260s 501ms/step - loss: 0.7345
Epoch 11/20
519/519 [==============================] - 260s 501ms/step - loss: 0.7107
Epoch 12/20
519/519 [==============================] - 260s 501

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 256)            31488     
_________________________________________________________________
gru_12 (GRU)                 (1, None, 1024)           3938304   
_________________________________________________________________
gru_13 (GRU)                 (1, None, 1024)           6297600   
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_6 (Dense)              (1, None, 123)            126075    
Total params: 18,786,171
Trainable params: 18,786,171
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
generated_text= generate_text(model, start_string=u"أمريكا تقرر")
print(generated_text)


أمريكا تقرر شطب 10 من أجزائر
ليبرمان: لا سبيل للرياضة التونسية
طائرة بين فتح وحماس بالضفة
"العنقفيو".. "عبور" دولي للتتقدم بعرقلة العرب
نهب بصعدة والنهضة تدخل المتشددين بتعز
رمضان في البحرين.. تنسيق سوريا
أقدمام سلاح يوم
توافق مصري على فتح معبر رفح
سوريا تتمسك بالتحالف مع شارون
مصر تضيف 3 غيغاواتات كأس العالم
شعبية بورصة قطر تخفض للنصف
الحكومة العراقية تنسحب من كابل
إنقاذ قبرص يتوقع انقلاب حسن
146 هديفة
البشير: التقيترتي إن أكبر سوق للطيران
جبهة انتصرت على نفط سياسية
الأزمة الكورية.. حين مطلع الأيرباص
عدل مبارك بنتائج قمح آيفون
الهاون بدلا من الحجارة للقاهرة
المعوم رياضي لحظر النقاب كليا
مشروع قرار "مخفف" نها
حين تخشى قراءة صادرات ألمانيا
عودة ظهور تراجع السيسي للأمير
هل الوساطة
قمة 50 كتائبيا على النيل
الرك برفع الحصار على iOS غزة
استهدفتان
المقاومة تسترد مناطق قرية صينية
ماذا تعرف عن فريق الجزيرة؟
ثلاثة آلاف عراقي يهز يراقب الموساد
دعوة لحوار سياسي في سوريا
المغرب يخسر: قوى إعمار غزة جيد متواصل
جوبا ترفض شروط عباس للتفاوض
دواء للنساء أكبر سوق للأبناء
منف بجوم جنوب السودان
بوش يروج لس

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

#**Use perplexity (PPL) for evaluation:**

Perplexity is the inverse probability of the test set, normalized by the number of words. 

[Reference](https://stackoverflow.com/a/33269399)

In [0]:
test_set = len(generated_text.split("\n"))
range(test_set)

range(0, 36)

In [0]:
import random
r = random.sample(range(test_set), 15)
test_set_sample = [generated_text.split("\n")[i] for i in r]
test_set_sample[0:10]


['قمة 50 كتائبيا على النيل',
 'ماذا تعرف عن فريق الجزيرة؟',
 'نهب بصعدة والنهضة تدخل المتشددين بتعز',
 'أقدمام سلاح يوم',
 'مصر تضيف 3 غيغاواتات كأس العالم',
 'الهاون بدلا من الحجارة للقاهرة',
 'دعوة لحوار سياسي في سوريا',
 'جبهة انتصرت على نفط سياسية',
 '146 هديفة',
 'الحكومة العراقية تنسحب من كابل']

In [0]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
import collections, nltk
# we first tokenize the text corpus
tokens = nltk.word_tokenize(text)

#here you construct the unigram language model 
def unigram(tokens):    
    model = collections.defaultdict(lambda: 0.01)
    for f in tokens:
        try:
            model[f] += 1
        except KeyError:
            model [f] = 1
            continue
    N = float(sum(model.values()))
    for word in model:
        model[word] = model[word]/N
    return model

In [0]:
#computes perplexity of the unigram model on a testset  
def perplexity(testset, model):
    testset= nltk.word_tokenize(testset)
    #testset = testset.split()
    perplexity = 1
    N = 0
    for word in testset:
        N += 1
        perplexity = perplexity * (1/model[word])
    perplexity = pow(perplexity, 1/float(N)) 
    return perplexity

In [0]:
model = unigram(tokens)

perplexities = []

for i in range(len(test_set_sample)):
    pp= perplexity(test_set_sample[i], model)
    print(pp)
    perplexities.append(pp)


733.8390326905856
948.7017526025071
18227.611014097427
972.675738451038
5326.319884173814
20507.98651320578
754.4113372639471
4437.294304579604
2620.2810127958983
2222.0081956417184
1274.4376250229634
5530.584865908439
2412.40911381818
14793.27266366582
1589.0682818303212


In [0]:
np.mean(pp)

1589.0682818303212

Note: 


* Perplexity is undefined if the language model assigns any zero probabilities to the test set. In that case your code should return positive infinity - float('inf').
* When dealing with perplexity, we try to reduce it. A language model that has less perplexity with regards to a certain test set is more desirable than one with a bigger perplexity.